# Biopython for working with PDB files

Useful links:

- Ultimate tutorial on Biopython: http://biopython.org/DIST/docs/tutorial/Tutorial.html 
- Navigation through documentation, case examples: https://biopython.org/wiki/Documentation
- Manual on PDB module: https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ

## 0. Installation 

In [ ]:
!pip install biopython

## 1. Parse PDB file into structure object

In [ ]:
!wget https://files.rcsb.org/download/1brs.pdb

In [ ]:
# Create PDBparser object
from Bio.PDB.PDBParser import PDBParser
p = PDBParser(QUIET=True) # silence warnings

In [ ]:
# Create the structure object
filename = '1brs.pdb'
# The first argument is your custom name for the structure. It is rarely handy, so you may just leave it empty.
structure = p.get_structure(' ', filename)

## 2. PDB file header

In [ ]:
# self-explanatory keys of header dictionary
structure.header.keys()

In [ ]:
# method of structure determination
structure.header['structure_method']

In [ ]:
# print missing residues (chain, residue name, residue number) if any
if structure.header['has_missing_residues'] == True:
    missres = structure.header['missing_residues']
    for res in missres:
        print(res['chain'], res['res_name'], res['ssseq'])

## 3. Coordinate section

Hierarchy of elemets in structure object:

- A structure consists of models. In X-Ray and EM structures there is, as a rule, one model, in NMR - multiple models (conformers).
- A model consists of chains. Chains are usually named by a single capital letter.
- A chain consists of residues
- A residue consists of atoms

In [ ]:
for model in structure:
    print(model)
    for chain in model:
        print(chain)
        for residue in chain:
            print(residue)
            for atom in residue:
                print(atom)

The nested for-loops in the code above are only for demonstration of structure object organization. In practice it is not convenient to iterate through all entities one by one. Accessing structure components by methods is more efficient:

In [ ]:
atoms = structure.get_atoms()

for atom in atoms:
    print(atom.get_coord()) # get atom coordinates

In [ ]:
for res in structure.get_residues():
    print(res.get_full_id(), res.get_resname()) 

Useful tip: to check methods and attributes of the objects, use `dir()`:

In [ ]:
for res in structure.get_residues():
    print(dir(res)) 
    break

Alternative way to access structure components is slicing:

In [ ]:
# chain A of the first model
chainA = structure[0]['A']

for atom in chainA.get_atoms():
    print(atom)
    break

In [ ]:
# residue with the residue number 4 of chain F
# mind that it is the residue insertion code in PDB file not python index
res4 = structure[0]['F'][4]
res4

In [ ]:
res4.get_full_id() 

The full id of the residue is a tuple with the following items:
    
0. structure name, that you specified when loading it using parser
1. model id
2. chain id
3. tuple, where the 2nd element is the number of the residue

In [ ]:
res4.get_full_id()[3][1]

## 4. Save structure object to PDB file

In [ ]:
from Bio.PDB.PDBIO import PDBIO
io=PDBIO()

Specify the structure or the part of it you want to save in `.set_structure()`.

In [ ]:
structure_to_save = structure[0]['F']
io.set_structure(structure_to_save)
name = '1brsF.pdb'
io.save(name)

## 5. Extract sequence from structure

Extract the sequence either from SEQRES record (`pdb-seqres`) or from ATOM record (`pdb-atom`). 

Sequence in SEQRES record is the sequence of the studied protein, while sequence from ATOM record is what was actually captured in the crystallographic experiment (if talking about X-ray determined structures). Some residues from SEQRES might be absent in the ATOM record since they are not resolved because of high flexibility or flaws of the experiment. SEQRES sequence may also differ from the sequence in reference databases, e.g. Uniprot. It might happen because researchers introduced mutations into the protein either to increase its stability in order to obtain a nice crystal for structure determination or to study the structure of this particular mutant.

Read more about the differences in SEQRES and ATOM sequences:

https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/primary-sequences-and-the-pdb-format \
https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html#SEQRES

More examples with `SeqIO` module: https://biopython.org/docs/1.75/api/Bio.SeqIO.PdbIO.html

In [ ]:
from Bio import SeqIO

In [ ]:
for record in SeqIO.parse(filename, "pdb-seqres"):
    print(record.annotations['chain'], record.seq)

In [ ]:
for record in SeqIO.parse(filename, "pdb-atom"):
    print(record.annotations['chain'], record.seq)

In [ ]:
for record in SeqIO.parse(filename, "pdb-atom"):
    if record.annotations['chain'] == 'C':
        seq = record.seq
        print(seq)

## 6. Save sequence to fasta file

In [ ]:
for record in SeqIO.parse(filename, "pdb-seqres"):
    if record.annotations['chain'] == 'A':
        fasta = record
        print(record.seq)

In [ ]:
SeqIO.write(fasta, "1brs_A.fasta", "fasta")

## 7. More sequence-related data

Apart from sequence, `SeqIO.parse()` outputs other useful information:

In [ ]:
for record in SeqIO.parse("1brs.pdb", "pdb-seqres"):
    print('What is record:', dir(record))
    print('Annotations:', record.annotations.keys())
    break

In [ ]:
for record in SeqIO.parse("1brs.pdb", "pdb-atom"):
    print('What is record:', dir(record))
    print('Annotations:', record.annotations.keys())
    break

In [ ]:
# get uniprot accession number
for record in SeqIO.parse(filename, "pdb-seqres"):
    if record.annotations['chain'] == 'A':
        ref = record.dbxrefs[0]
        print(ref)